In [64]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
from main import read_file, get_sentences
import stanza

stanza.download("en", model_dir="../models/stanza_resources")

2023-04-24 17:50:06 INFO: Downloading default packages for language: en (English) ...
04/24/2023 17:50:06 - INFO - 	 Downloading default packages for language: en (English) ...
2023-04-24 17:50:08 INFO: File exists: ../models/stanza_resources/en/default.zip
04/24/2023 17:50:08 - INFO - 	 File exists: ../models/stanza_resources/en/default.zip
2023-04-24 17:50:11 INFO: Finished downloading models and saved to ../models/stanza_resources.
04/24/2023 17:50:11 - INFO - 	 Finished downloading models and saved to ../models/stanza_resources.


In [66]:
henry_red = read_file("../data/english_short_stories/LeiningenVstheAnts.txt")

preprocess = stanza.Pipeline("en", processors="tokenize, ner, pos, lemma")

doc = preprocess(henry_red)

2023-04-24 17:50:11 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
04/24/2023 17:50:11 - INFO - 	 Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-04-24 17:50:12 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| ner       | ontonotes |

04/24/2023 17:50:12 - INFO - 	 Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| ner       | ontonotes |

2023-04-24 17:50:12 INFO: Using device: cpu
04/24/2023 17:50:12 - INFO - 	 Using device: cpu
2023-04-24 17:50:12 INFO: Loading: tokenize
04/24/2023 17:50:12 - INFO - 	 Loading: tokenize
2023-04-24 17:50:12 INFO: Loading: pos
04/24/2023 17:50:12 - INFO - 	 Loading: pos
2023-04-24 17:50:12 INFO: Loading: lemma
04/24/2023 17:50:12 - INFO - 	 Loading: lemma
2023-04-24 17:50:12 INFO: Loading: ner
04/24/2023 17:50:12 - INFO - 	 Loading: ner
2023-04-24 17:50:13 INFO: Done loading processors!
04/24/2023 17:50:13 - INFO - 	 Done loadi

In [67]:
sentences = get_sentences(doc, lambda x: x.upos != "PUNCT")

In [68]:
from main import get_named_entities
named_entities = get_named_entities(doc)

names = []
for ne in named_entities:
    for named_entity in ne:
        names.append(named_entity.text)

from collections import Counter

name_frequencies = Counter(names)
name_frequencies = {k:v for (k,v) in sorted(name_frequencies.items(), key=lambda item: item[1], reverse=True)}
name_frequencies

{'Leiningen': 51,
 'Carl Stephenson': 1,
 'Hood': 1,
 'Dullards': 1,
 'Pell-mell': 1,
 'Dawn': 1,
 'Macbeth': 1,
 'Birnam Wood': 1,
 'Scot': 1,
 'Dante': 1,
 'Spry': 1,
 'Swift': 1}

In [69]:
# Coreference resolution
from fastcoref import FCoref
model = FCoref()
# henry_red = read_file("../data/english_short_stories/Henry_Red_Chief.txt")

preds = model.predict(henry_red)
clusters = preds.get_clusters(as_strings=False)
clusters_with_names = preds.get_clusters()

04/24/2023 17:50:37 - INFO - 	 missing_keys: []
04/24/2023 17:50:37 - INFO - 	 unexpected_keys: []
04/24/2023 17:50:37 - INFO - 	 mismatched_keys: []
04/24/2023 17:50:37 - INFO - 	 error_msgs: []
04/24/2023 17:50:37 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
04/24/2023 17:50:37 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

04/24/2023 17:50:38 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

In [70]:
import spacy
from fastcoref import spacy_component
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('fastcoref')

# This is better model
# nlp.add_pipe("fastcoref", config={'model_architecture': 'LingMessCoref', 'model_path': 'biu-nlp/lingmess-coref', 'device': 'cpu'})

04/24/2023 17:50:44 - INFO - 	 missing_keys: []
04/24/2023 17:50:44 - INFO - 	 unexpected_keys: []
04/24/2023 17:50:44 - INFO - 	 mismatched_keys: []
04/24/2023 17:50:44 - INFO - 	 error_msgs: []
04/24/2023 17:50:44 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M


In [71]:
# henry_red = read_file("../data/english_short_stories/Henry_Red_Chief.txt")

# This substitutes pronouns with appropriate Named entities
coref_doc = nlp(henry_red, component_cfg={'fastcoref': {'resolve_text': True}})
resolved_text = coref_doc._.resolved_text

# Again perform NER on new text to get all the positions of the NE.
ne_doc = preprocess(resolved_text)
corrected_sentences = get_sentences(ne_doc, lambda x: x.upos != "PUNCT")
total_named_entities = get_named_entities(ne_doc)
total_names = []
for ne in total_named_entities:
    for named_entity in ne:
        total_names.append(named_entity.text)

from collections import Counter

total_name_frequencies = Counter(total_names)
total_name_frequencies = {k:v for (k,v) in sorted(total_name_frequencies.items(), key=lambda item: item[1], reverse=True)}

keys_to_delete = []
for name in total_name_frequencies:
    for name2 in total_name_frequencies:
        if name == name2[:-2]:
            total_name_frequencies[name] += total_name_frequencies[name2]
            keys_to_delete.append(name2)
        if name in name2 and name != name2:
            total_name_frequencies[name] += total_name_frequencies[name2]
            keys_to_delete.append(name2)
for k in keys_to_delete:
    if k in total_name_frequencies:
        total_name_frequencies.pop(k) 
total_name_frequencies


04/24/2023 17:50:46 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

04/24/2023 17:50:47 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

{'Leiningen': 300,
 'Macbeth': 3,
 'Carl Stephenson': 1,
 'Hood': 1,
 'Pell-mell': 1,
 'Dawn': 1,
 'Birnam Wood': 1,
 'Dante': 1,
 'Spry': 1,
 'Swift': 1}